Usando los dataframe de Dask, prepararemos un notebook para desarrollar los siguientes
puntos

> Cargaremos el conjunto de datos en un dataframe

In [73]:
import dask.dataframe as dd

In [74]:
df1 = dd.read_csv("data/air_traffic_data.csv")
df2 = dd.read_csv("data/ABT_CALIDAD_AIRE.csv")
df = dd.merge(df1, df2)
df.head()

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,...,chance_of_rain,will_it_snow,chance_of_snow,vis_km,festivo,anyo,mes,dia,hora,diasemana
0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,0,1
1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,1,1
2,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,2,1
3,200507,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,Terminal 1,...,0,0,0,10.0,0,2017,10,1,3,1
4,200507,Air Canada,AC,Air Canada,AC,International,Canada,Enplaned,Other,Terminal 1,...,0,0,0,10.0,0,2017,10,1,4,1


He metido todos los datos en un mismo dataframe

> ¿Cuántas compañías diferentes aparecen en el fichero?

In [75]:
df["Operating Airline"].unique().compute()

0            ATA Airlines
1             Air Canada 
2               Air China
3              Air France
4         Air New Zealand
             ...         
72         Etihad Airways
73         China Southern
74       Turkish Airlines
75    COPA Airlines, Inc.
76      Air India Limited
Name: Operating Airline, Length: 77, dtype: object

Aparecen 77 compañías diferentes

> ¿Cuántos pasajeros tienen de media los vuelos de cada compañía?

In [76]:
df.groupby("Operating Airline")["Adjusted Passenger Count"].mean().compute()

Operating Airline
ATA Airlines          9661.659091
Aer Lingus            4407.183673
Aeromexico            5463.822222
Air Berlin            2320.750000
Air Canada           18251.560109
                         ...     
Virgin Atlantic       9847.104651
WestJet Airlines      5338.155340
World Airways          261.666667
XL Airways France     2240.129032
Xtra Airways            73.000000
Name: Adjusted Passenger Count, Length: 77, dtype: float64

Es creible porque según los datos de esa parte del dataset, el conteo de pasajeros por vuelo es muy alto

> Eliminaremos los registros duplicados por el campo “GEO Región”, manteniendo
únicamente aquel con mayor número de pasajeros.

In [77]:
def shaper(df):
    return f"({df.shape[0].compute()}, {df.shape[1]})"

In [78]:
shaper(df)

'(15007, 102)'

In [79]:
geo_df = df.sort_values(by=["Adjusted Passenger Count"], ascending=False)
geo_df = geo_df.reset_index(drop=True)
geo_df = geo_df.drop_duplicates(subset=["GEO Region"], keep="first")
geo_df = geo_df.reset_index(drop=True)
geo_df.head()

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,...,chance_of_rain,will_it_snow,chance_of_snow,vis_km,festivo,anyo,mes,dia,hora,diasemana
0,201308,United Airlines,UA,United Airlines,UA,Domestic,US,Deplaned,Other,Terminal 3,...,0,0,0,10.0,0,2017,11,18,3,7
1,200708,United Airlines - Pre 07/01/2013,UA,United Airlines - Pre 07/01/2013,UA,International,Asia,Deplaned,Other,International,...,0,0,0,10.0,0,2018,2,1,21,5
2,201507,United Airlines,UA,United Airlines,UA,International,Europe,Deplaned,Other,International,...,63,0,0,6.3,0,2018,3,9,16,6
3,200708,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,Terminal 3,...,1,0,0,10.0,0,2018,1,28,16,1
4,201407,United Airlines,UA,United Airlines,UA,International,Mexico,Deplaned,Other,International,...,68,0,0,9.3,0,2018,1,9,14,3


In [80]:
shaper(geo_df)

'(9, 102)'

In [81]:
df["GEO Region"].unique().compute()

0                     US
1                 Canada
2                   Asia
3                 Europe
4    Australia / Oceania
5                 Mexico
6        Central America
7            Middle East
8          South America
Name: GEO Region, dtype: object

In [82]:
geo_df["GEO Region"].unique().compute()

0                     US
1                   Asia
2                 Europe
3                 Canada
4                 Mexico
5            Middle East
6    Australia / Oceania
7        Central America
8          South America
Name: GEO Region, dtype: object

In [83]:
reg=[]
as_pas=[]
for i in list(geo_df["GEO Region"]):
    reg.append(i)
for i in list(geo_df["Adjusted Passenger Count"]):
    as_pas.append(i)

In [84]:
for i in range(len(reg)):
    print(f"{reg[i]}:  {as_pas[i]}")

US:  659837
Asia:  86398
Europe:  48136
Canada:  39798
Mexico:  29206
Middle East:  14769
Australia / Oceania:  12973
Central America:  8970
South America:  3685


Los maximos oficiales del dataframe completo

In [85]:
for col in reg:
    print(f"{col}: {df[df['GEO Region']==col]['Adjusted Passenger Count'].max().compute()}")

US: 659837
Asia: 86398
Europe: 48136
Canada: 39798
Mexico: 29206
Middle East: 14769
Australia / Oceania: 12973
Central America: 8970
South America: 3685


Nos hemos cargado los datos totalmente, porque hemmos pasado de 15007 filas a 9, lo que quiere decir que solo existen 9 tipos diferentes de `"GEO Region"` 

> Volcaremos los resultados de los dos puntos anteriores a un CSV

In [72]:
medias = []
for col in reg:
    medias.append(df[df['GEO Region']==col]['Adjusted Passenger Count'].mean().compute())

In [88]:
dict_final = {"GEO Region": reg, "Adjusted Passenger Count": as_pas, "Means":medias}

In [90]:
import pandas as pd

In [91]:
final_df = pd.DataFrame(dict_final)
final_df.head()

,GEO Region,Adjusted Passenger Count,Means
0,US,659837,58485.878385
1,Asia,86398,13508.552704
2,Europe,48136,12779.055050
3,Canada,39798,9803.791255
4,Mexico,29206,7250.898655


In [92]:
final_df.to_csv("data/geo_data.csv", index=False)